# Segmenting and Clustering Neighborhoods in Toronto

## Getting Toronto neighborhoods

In [1]:
!conda install -yc conda-forge geocoder lxml cssselect
import pandas as pd
import numpy as np
import geocoder 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - cssselect
    - geocoder
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    click-7.0                  |             py_0      

*I use pandas to get data frames from a we page. Then I remove entries that have "Not assigned" in borough name and then replace ones in neighborhood names with borough name*

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df = df[df['Borough'] != 'Not assigned']
df[df['Neighborhood'] == 'Not assigned'] = df['Borough']
df = df.reset_index(drop = True)
df.shape

(210, 3)

*I use Bing API to get geocodindg of neigborhoods by their names.*

In [3]:
import time
BING_KEY = 'Aq7bYrs0DLI7HUs7AbCd0Ejxsr72-c_RVKm0t1p6ivr0Dne8fbEdie88uM9rQlvS'
def get_location(zip):
    #time.sleep(1)
    geo = geocoder.bing("{}, Toronto, Canada".format(zip), key = BING_KEY)
    #print(geo.json)
    return pd.Series(geo.latlng, index = ('Latitude', 'Longitude'))


In [4]:
neighborhoods = df.join(df['Postcode'].apply(lambda n: get_location(n)))

In [5]:
neighborhoods

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.751255,-79.329895
1,M4A,North York,Victoria Village,43.729958,-79.314201
2,M5A,Downtown Toronto,Harbourfront,43.655220,-79.361969
3,M6A,North York,Lawrence Heights,43.722801,-79.450691
4,M6A,North York,Lawrence Manor,43.722801,-79.450691
...,...,...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West,43.625164,-79.526550
206,M8Z,Etobicoke,Mimico NW,43.625164,-79.526550
207,M8Z,Etobicoke,The Queensway West,43.625164,-79.526550
208,M8Z,Etobicoke,Royal York South West,43.625164,-79.526550


## Explore neighborhoods

In [6]:
import folium # map rendering library
import requests

In [7]:
address = 'Toronto, ON'

location = geocoder.bing(address, key = BING_KEY)
latitude, longitude = location.latlng
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.64868927001953, -79.38543701171875.


In [8]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [9]:
CLIENT_ID = 'IHA2RZ534UBEJJ0OX0X3HCPDS4HQL4TQTCUHCYPEW3XLQDO3' # your Foursquare ID
CLIENT_SECRET = 'LASEJ4FGP0SZSRYNE4I3YE2MCWI2YWAIAAPSUX54EPELLAFD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IHA2RZ534UBEJJ0OX0X3HCPDS4HQL4TQTCUHCYPEW3XLQDO3
CLIENT_SECRET:LASEJ4FGP0SZSRYNE4I3YE2MCWI2YWAIAAPSUX54EPELLAFD


In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('{}... '.format(name), end = '')
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        print("found {} venues.".format(len(results)))

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
# As there is a limitation of API calls to Foursquare I just saved it to CSV and use it offline
#tmp = neighborhoods.sort_values(['Neighborhood', 'Borough'])
#toronto_venues = getNearbyVenues(names=tmp['Neighborhood'],
#                                   latitudes=tmp['Latitude'],
#                                   longitudes=tmp['Longitude'],
#                                   limit = 100
#                                  )
#toronto_venues.to_csv('toronto-venues.csv', index = False)

In [13]:
toronto_venues = pd.read_csv('toronto-venues.csv')
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adelaide,43.650410,-79.382881,The Keg Steakhouse & Bar,43.649937,-79.384196,Steakhouse
1,Adelaide,43.650410,-79.382881,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,Adelaide,43.650410,-79.382881,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
3,Adelaide,43.650410,-79.382881,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center
4,Adelaide,43.650410,-79.382881,Cactus Club Cafe,43.649552,-79.381671,American Restaurant
...,...,...,...,...,...,...,...
4546,Yorkville,43.674583,-79.401741,Martino's Pizza,43.675560,-79.403558,Pizza Place
4547,Yorkville,43.674583,-79.401741,Tim Hortons,43.675800,-79.403532,Coffee Shop
4548,Yorkville,43.674583,-79.401741,Subway® Restaurants,43.675071,-79.406877,Sandwich Place
4549,Yorkville,43.674583,-79.401741,Nick's Shoe Repair,43.675457,-79.403675,Shoe Repair


In [14]:
# Some venues have "Neighborhood" category and refer to "Downtown Toronto". I'll just remove them
toronto_venues = toronto_venues[toronto_venues['Venue Category'] != 'Neighborhood']

In [15]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,99,99,99,99,99,99
Agincourt,19,19,19,19,19,19
Agincourt North,4,4,4,4,4,4
Albion Gardens,12,12,12,12,12,12
Alderwood,8,8,8,8,8,8
...,...,...,...,...,...,...
Woodbine Heights,12,12,12,12,12,12
York Mills,1,1,1,1,1,1
York Mills West,4,4,4,4,4,4


In [16]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 262 uniques categories.


In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.shape

(4547, 263)

In [18]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.0,0.0,0.020202,0.010101,0.0,0.000000,0.030303,0.0,0.0,...,0.0,0.010101,0.0,0.000000,0.000000,0.010101,0.0,0.0,0.010101,0.0
1,Agincourt,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.052632,0.000000,0.0,0.0,0.000000,0.0
2,Agincourt North,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
3,Albion Gardens,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.083333,0.000000,0.000000,0.0,0.0,0.000000,0.0
4,Alderwood,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,Woodbine Heights,0.0,0.0,0.000000,0.000000,0.0,0.083333,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
201,York Mills,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
202,York Mills West,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
203,York University,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0


#### First, let's write a function to sort the venues in descending order

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Cluster neighborhoods

In [20]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [37]:
# set number of clusters
kclusters = 2

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print("Got {} labels for {} neighborhoods".format(len(kmeans.labels_), len(toronto_grouped)))

Got 205 labels for 205 neighborhoods


In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,Restaurant,Hotel,Bar,Asian Restaurant,Burger Joint,Gastropub,Breakfast Spot
1,Agincourt,Chinese Restaurant,Coffee Shop,Pizza Place,Badminton Court,Shanghai Restaurant,Bubble Tea Shop,Sandwich Place,Supermarket,Sushi Restaurant,Skating Rink
2,Agincourt North,Pharmacy,Park,Gym,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Yoga Studio
3,Albion Gardens,Grocery Store,Pizza Place,Park,Caribbean Restaurant,Fried Chicken Joint,Beer Store,Sandwich Place,Fast Food Restaurant,Hardware Store,Video Store
4,Alderwood,Pizza Place,Gas Station,Coffee Shop,Pub,Gym,Pharmacy,Convenience Store,Sandwich Place,Farm,Falafel Restaurant


In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
# and then we remove neighborhoods that have no venues at all (cluster label and most common venues are NaN)
toronto_merged = toronto_merged.dropna()

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Now, we examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we then assign a name to each cluster.

#### Cluster 1

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]["1st Most Common Venue"].unique()

array(['Food & Drink Shop', 'Park', 'Gym / Fitness Center',
       'Convenience Store', 'Accessories Store', 'Playground', 'Pharmacy'],
      dtype=object)

#### Cluster 2

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]["1st Most Common Venue"].unique()

array(['Pizza Place', 'Coffee Shop', 'Clothing Store',
       'Fast Food Restaurant', 'Mediterranean Restaurant', 'Bar',
       'Asian Restaurant', 'Pet Store', 'Playground', 'Shopping Mall',
       'Mexican Restaurant', 'Business Service', 'Sporting Goods Shop',
       'Café', 'Construction & Landscaping', 'Chinese Restaurant',
       'Trail', 'Park', 'Train Station', 'Massage Studio', 'Theme Park',
       'Convenience Store', 'Soccer Field', 'Grocery Store',
       'Middle Eastern Restaurant', 'Home Service', 'Sandwich Place',
       'Swim School', 'Garden', 'Brewery', 'Gift Shop',
       'Food & Drink Shop', 'Italian Restaurant',
       'Eastern European Restaurant', 'American Restaurant', 'Gym',
       'Doner Restaurant', 'Burrito Place'], dtype=object)

**Examination of the clusters shows there are 2 neighborhood clusters that can be distinguished:**
 - Cluster 1 - Can be called *residential* one. 
 - Cluster 2 - Can be called *business and service* one